In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import matplotlib.pyplot as pl

In [3]:
train_path = "/content/drive/MyDrive/Colab Notebooks/Xray_Classifier/Xray_Classifier/train"
test_path = "/content/drive/MyDrive/Colab Notebooks/Xray_Classifier/Xray_Classifier/train"
val_path = "/content/drive/MyDrive/Colab Notebooks/Xray_Classifier/Xray_Classifier/val"

In [4]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory = train_path, target_size = (150,150),batch_size = 32)
val_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=val_path, target_size=(150,150), batch_size=32)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(150,150), batch_size=32, shuffle=False)

Found 1185 images belonging to 2 classes.
Found 1185 images belonging to 2 classes.
Found 1185 images belonging to 2 classes.


In [5]:
mobile = tf.keras.applications.mobilenet.MobileNet()
mobile.summary()

17225924/17225924 [==============================] - 0s 0us/step
Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                 

In [6]:
x = mobile.layers[-2].output
output = Dense(2, activation = "sigmoid")(x)
model = Model(inputs = mobile.input, outputs = output)

In [7]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics =["accuracy"])

In [8]:
es = keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='auto', restore_best_weights=True, verbose=1, patience=7)

In [9]:
model.fit(x = train_batches,steps_per_epoch=len(train_batches),validation_data=val_batches, validation_steps=len(val_batches), epochs=100, callbacks=[es], verbose =1)

Epoch 1/100
38/38 [==============================] - 508s 13s/step - loss: 0.2071 - accuracy: 0.9722 - val_loss: 0.1835 - val_accuracy: 0.9899
Epoch 2/100
38/38 [==============================] - 17s 452ms/step - loss: 0.0234 - accuracy: 0.9958 - val_loss: 0.3038 - val_accuracy: 0.9941
Epoch 3/100
38/38 [==============================] - 17s 446ms/step - loss: 0.0300 - accuracy: 0.9992 - val_loss: 2.0775 - val_accuracy: 0.9873
Epoch 4/100
38/38 [==============================] - 17s 436ms/step - loss: 0.0506 - accuracy: 0.9916 - val_loss: 0.0101 - val_accuracy: 1.0000
Epoch 5/100
38/38 [==============================] - 16s 430ms/step - loss: 0.0044 - accuracy: 0.9983 - val_loss: 0.0102 - val_accuracy: 0.9983
Epoch 6/100
38/38 [==============================] - 16s 435ms/step - loss: 3.7745e-04 - accuracy: 1.0000 - val_loss: 6.0273e-04 - val_accuracy: 1.0000
Epoch 7/100
38/38 [==============================] - 16s 429ms/step - loss: 0.3441 - accuracy: 0.9806 - val_loss: 0.6931 - val_ac

In [10]:
test_labels = test_batches.classes
predictions = model.predict(test_batches, steps = len(test_batches),verbose = 0)

In [12]:
from sklearn.metrics import accuracy_score
predicted_labels = np.argmax(predictions, axis=1)

accuracy = accuracy_score(test_labels, predicted_labels)
print("Modelin doğruluğu:", accuracy)

Modelin doğruluğu: 1.0
